In [1]:
import pandas as pd
import json
from utils.constants import OBJECTIVES_SUDOKU
from utils.utils import load_sudokus_from_json, create_folders
from utils.utils_classes import Oracle
from utils.utils_training import PreferenceElicitationFramework
from utils.utils_classes import Human
from utils.utils_sudoku import generate_steps_sudoku
import ast
import numpy as np
from tqdm import tqdm
from utils.utils import count_and_remove_matching_hints

In [2]:
lr = 0.5
initial_weights = dict.fromkeys(OBJECTIVES_SUDOKU, 1)
normalization = 2
steps = 'SMUS'

In [3]:
username = input("Enter your username: ")
if not username.strip():
    print("Username is required!")
else:
    print(f"Username entered: {username}")

Enter your username:  marco


Username entered: marco


In [4]:
sudokus = load_sudokus_from_json()

steps_sudokus = []
for index in range(15):
    steps_single_sudoku = []
    df_steps = pd.read_csv(f'data/gt_sudoku/SMUS/sudoku_user_SMUS_sudoku_{index}.csv',index_col=False)
    for index, row in df_steps.iterrows():
        grid = np.fromstring(row['grid'].replace("[", "").replace("]", ""), sep=" ").astype(int).reshape(9, 9)
        r,c = ast.literal_eval(row['hint_derived'])[1][:2]
        steps_single_sudoku.append([grid,[r,c]])
    steps_sudokus.append(steps_single_sudoku)


weights = dict.fromkeys(OBJECTIVES_SUDOKU, 1)



oracle = Human()
output_location_machop = f'results/sudoku/real_case_{username}_{lr}_MACHOP/'
output_location_baseline = f'results/sudoku/real_case_{username}_{lr}_baseline/'

In [6]:
user = 0
df_steps_evaluation = ''
pef = PreferenceElicitationFramework(logic_puzzles_set=sudokus, oracle=oracle, no_oracle=user,
                                     initial_weights=initial_weights, instance_evaluation=sudokus[30],
                                     df_steps_evaluation=df_steps_evaluation, output_location=output_location_machop,
                                     time_eval=150,max_data=50,
                                     normalized=normalization, batch_size=1, frozen_steps=steps_sudokus,
                                     lr=lr, type_diversification='cpucb_disjunction', custom_tuner=None,
                                     exploration_root=1)
learned_weights  = pef.start()

Labelling progress: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [01:52<00:00,  2.25s/it]


In [7]:
user = 0
df_steps_evaluation = ''
pef = PreferenceElicitationFramework(logic_puzzles_set=sudokus, oracle=oracle, no_oracle=user,
                                     initial_weights=initial_weights, instance_evaluation=sudokus[30],
                                     df_steps_evaluation=df_steps_evaluation, output_location=output_location_baseline,
                                     time_eval=150,max_data=50,
                                     normalized=normalization, batch_size=1, frozen_steps=steps_sudokus,
                                     lr=lr, type_diversification='baseline', custom_tuner=None,
                                     exploration_root=1)
learned_weights  = pef.start()

Labelling progress: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [02:55<00:00,  3.51s/it]


In [12]:
def get_weights_norm_steps(step):
    df_weights_machop = pd.read_csv(f'{output_location_machop}weights.csv')
    df_norm_machop = pd.read_csv(f'{output_location_machop}norm.csv')


    weights_column = df_weights_machop['weights']
    last_value_str = weights_column.iloc[step-1]
    weights_machop = ast.literal_eval(last_value_str)

    weights_column = df_norm_machop['norm']
    last_value_str = weights_column.iloc[step-1]
    norm_machop = ast.literal_eval(last_value_str)



    df_weights_baseline = pd.read_csv(f'{output_location_baseline}weights.csv')
    df_norm_baseline = pd.read_csv(f'{output_location_baseline}norm.csv')

    weights_column = df_weights_baseline['weights']
    last_value_str = weights_column.iloc[step-1]
    weights_baseline = ast.literal_eval(last_value_str)

    weights_column = df_norm_baseline['norm']
    last_value_str = weights_column.iloc[step-1]
    norm_baseline = ast.literal_eval(last_value_str)

    return weights_machop,norm_machop,weights_baseline,norm_baseline


In [13]:
for step in [10,30,50]:
    weights_machop,norm_machop,weights_baseline,norm_baseline = get_weights_norm_steps(step)
    df_steps_evaluation = pd.read_csv('data/gt_sudoku/sudoku_gt_SMUS_sudoku_30.csv')
    predicted = []
    for index, row in tqdm(df_steps_evaluation.iterrows(), total=len(df_steps_evaluation)):
        sudoku_eval = row['grid'].replace(' ', ', ')
        sudoku_eval = np.array(eval(sudoku_eval))
        parsed_list = eval(row['hint_derived'])
        explained_row = parsed_list[1][0]
        explained_col = parsed_list[1][1]
        exp,_,_ = generate_steps_sudoku(to_return=True, grid=sudoku_eval,
                                        feature_weights=weights_machop,no_user=1,
                                        sequential_sudoku=-1,normalization=norm_machop,
                                        is_tqdm=False)
        predicted.append(exp)

    df_predicted_steps = pd.DataFrame(predicted)
    df_predicted_steps.to_csv(f'results/sudoku/real_case_{username}_{lr}_MACHOP/machop_SMUS_{step}.csv', index=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 56/56 [02:26<00:00,  2.61s/it]


In [14]:
for step in [10,30,50]:
    weights_machop,norm_machop,weights_baseline,norm_baseline = get_weights_norm_steps(step)
    df_steps_evaluation = pd.read_csv('data/gt_sudoku/sudoku_gt_SMUS_sudoku_30.csv')
    predicted = []
    for index, row in tqdm(df_steps_evaluation.iterrows(), total=len(df_steps_evaluation)):
        sudoku_eval = row['grid'].replace(' ', ', ')
        sudoku_eval = np.array(eval(sudoku_eval))
        parsed_list = eval(row['hint_derived'])
        explained_row = parsed_list[1][0]
        explained_col = parsed_list[1][1]
        exp,_,_ = generate_steps_sudoku(to_return=True, grid=sudoku_eval,
                                        feature_weights=weights_baseline,no_user=1,
                                        sequential_sudoku=-1,normalization=norm_baseline,
                                        is_tqdm=False)
        predicted.append(exp)

    df_predicted_steps = pd.DataFrame(predicted)
    df_predicted_steps.to_csv(f'results/sudoku/real_case_{username}_{lr}_baseline/baseline_SMUS_{step}.csv', index=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 56/56 [02:52<00:00,  3.09s/it]


In [5]:
dict_df = {}
for step in [10,30,50]:
    df_smus = pd.read_csv('data/gt_sudoku/sudoku_gt_SMUS_sudoku_30.csv')
    df_learnt_baseline = pd.read_csv(f'results/sudoku/real_case_{username}_{lr}_baseline/baseline_SMUS_{step}.csv')
    df_learnt_machop = pd.read_csv(f'results/sudoku/real_case_{username}_{lr}_MACHOP/machop_SMUS_{step}.csv')
    dict_df[step] = [df_smus, df_learnt_baseline, df_learnt_machop]

In [6]:
for step in [10,30,50]:
    df_smus,df_learnt_baseline,df_learnt_machop = dict_df[step]
    ties, df_smus_cleaned, df_learnt_machop_cleaned = count_and_remove_matching_hints(df_smus.copy(), df_learnt_machop.copy())
    res = [df_smus_cleaned,df_learnt_machop_cleaned]
    df_hand, df_learnt = res

    # Get a random permutation of indices
    shuffled_indices = np.random.permutation(len(df_hand))

    # Apply the shuffled indices to both DataFrames
    df_hand_shuffled = df_hand.iloc[shuffled_indices].reset_index(drop=True)
    df_learnt_shuffled = df_learnt.iloc[shuffled_indices].reset_index(drop=True)

    # Now you have shuffled DataFrames with the same row order
    res_shuffled = [df_hand_shuffled, df_learnt_shuffled]

    oracle = Human()
    res = oracle.label(res_shuffled,evaluation=True)
    res['identical'] = ties
    create_folders(f'results/sudoku/real_case_{username}/')
    with open(f'results/sudoku/real_case_{username}/SMUS_machop_{step}.json', "w") as json_file:
        json.dump(res, json_file, indent=4)

  2%|███▉                                                                                                                                                                                   | 1/47 [00:06<04:52,  6.37s/it]


ValueError: invalid literal for int() with base 10: 'a'

In [53]:
for step in [10,30,50]:
    df_smus,df_learnt_baseline,df_learnt_machop = dict_df[step]
    ties, df_smus_cleaned, df_learnt_baseline_cleaned = count_and_remove_matching_hints(df_smus.copy(), df_learnt_baseline.copy())
    res = [df_smus_cleaned,df_learnt_baseline_cleaned]
    df_hand, df_learnt = res

    # Get a random permutation of indices
    shuffled_indices = np.random.permutation(len(df_hand))

    # Apply the shuffled indices to both DataFrames
    df_hand_shuffled = df_hand.iloc[shuffled_indices].reset_index(drop=True)
    df_learnt_shuffled = df_learnt.iloc[shuffled_indices].reset_index(drop=True)

    # Now you have shuffled DataFrames with the same row order
    res_shuffled = [df_hand_shuffled, df_learnt_shuffled]

    oracle = Human()
    res = oracle.label(res_shuffled,evaluation=True)
    res['identical'] = ties
    create_folders(f'results/sudoku/real_case_{username}/')
    with open(f'results/sudoku/real_case_{username}/SMUS_baseline_{step}.json', "w") as json_file:
        json.dump(res, json_file, indent=4)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 56/56 [01:32<00:00,  1.65s/it]

PLEASE REPORT TO MARCO THE FOLLOWING
{'SMUS': 16, 'Learnt': 37, 'Draw': 3}
